In [1]:
import sqlite3
import pickle

protocol = 0
sqlite3.register_converter("pickle", pickle.loads)
sqlite3.register_adapter(list, pickle.dumps)
sqlite3.register_adapter(set, pickle.dumps)



table_name    = "articles"
insert_string = "INSERT into %s values (?, ?, ?)" % (table_name)
update_string = "UPDATE %s SET lines=?, parents=? WHERE id=?" % (table_name)
select_string = "SELECT id, lines, parents FROM %s WHERE id=?" % (table_name)

def create_schema(cursor):
    try:
        cursor.execute("""
            CREATE TABLE %s (
                id    integer primary key not null,
                lines pickle,
                parents pickle
                )""" % (table_name))
    except sqlite3.OperationalError:
        pass

def insert_into_db(cursor, obj):
    try:
        cursor.execute(
            insert_string,
            obj
        )
    except sqlite3.IntegrityError:
        print("Duplicate key")
    conn.commit()


def update_in_db(cursor, obj):
    cursor.execute(update_string,
        obj
    )

def get_obj_from_db(cursor, key):
    cursor.execute(select_string, (key))
    data = cursor.fetchone()
    return data


conn = sqlite3.connect('example.db', detect_types=sqlite3.PARSE_DECLTYPES)
c = conn.cursor()

create_schema(c)

print("created schema")

key = 1

some_object = (key, [1,2,3], set([1,2,3]))

print("created object")

insert_into_db(c, some_object)

print( get_obj_from_db(c, (key,)))

update_in_db(c, (key, [1,2,4], set([1,2,3])) )

print( get_obj_from_db(c, (key,)) )

print("saved object")

created schema
created object
(1, [1, 2, 3], {1, 2, 3})
(1, [1, 2, 3], {1, 2, 3})
saved object
